In [1]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.utils import to_categorical

!pip install scikit-plot
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets

In [3]:
X_split = np.array_split(X, 4)
y_split = np.array_split(y, 4)

trainX_d, testX_d, trainY_d, testY_d = train_test_split(X_split[3], y_split[3], test_size=0.2)

In [4]:
model_d = Sequential()
model_d.add(Dense(3, activation='relu', input_shape=(trainX_d.shape[1], 1)))
model_d.add(Dropout(0.1))
model_d.add(Flatten())
model_d.add(Dense(1, activation='sigmoid'))
model_d.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_d.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 21, 3)             6         
                                                                 
 dropout (Dropout)           (None, 21, 3)             0         
                                                                 
 flatten (Flatten)           (None, 63)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 64        
                                                                 
Total params: 70 (280.00 Byte)
Trainable params: 70 (280.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
history_d = model_d.fit(trainX_d, trainY_d, epochs=3)

Epoch 1/3
1586/1586 [==============================] - 9s 5ms/step - loss: 0.5016 - accuracy: 0.8190
Epoch 2/3
1586/1586 [==============================] - 9s 6ms/step - loss: 0.3601 - accuracy: 0.8546
Epoch 3/3
1586/1586 [==============================] - 13s 8ms/step - loss: 0.3546 - accuracy: 0.8539


In [6]:
score_d = model_d.evaluate(testX_d, testY_d, verbose=0)
print(score_d)

[0.3355019688606262, 0.8633711934089661]


In [ ]:
pesiTmp_d = model_d.get_weights()

In [7]:
import requests as r
import json

In [8]:
pesiTmp2_d = [pesiTmp_d[0][0], pesiTmp_d[1]]
# Conversione dei pesi per renderli compatibili con la codifica JSON e del server
pesi_d = '{\"l1\" : [[' + np.array2string(pesiTmp_d[0][0][0]) + '],[' + np.array2string(pesiTmp_d[0][0][1]) + '],[' + np.array2string(pesiTmp_d[0][0][2]) + ']], \"a\" : \"' + np.array2string(pesiTmp_d[1]) + '\"}'
pesi_json_d = json.loads(pesi_d)
print(pesi_d)

NameError: name 'pesiTmp_d' is not defined

In [ ]:
url = 'http://93.65.162.11:3001'
x = r.post(url + '/nodo_d', data = pesi_d, headers={"Content-Type": "application/json"})
print(x)

In [ ]:
import json
media = r.get(url + '/pesi')
dict = json.loads(media.text)

pesi_tmp = np.array(dict['l1'])
pesi_tmp_d = np.array(dict['a'])

pesi_nodi = pesi_tmp.reshape((1,3))
pesi_archi = pesi_tmp_d.reshape((1,3))
pesi_archi[pesi_archi == None] = 0

In [ ]:
pesi_tmp_d = model_d.get_weights()

#per renderlo malevol
pesi_tmp_d[0][0][0] = 500 #pesi_nodi[0][0]
pesi_tmp_d[0][0][1] = 500 #pesi_nodi[0][1]
pesi_tmp_d[0][0][2] = 500 #pesi_nodi[0][2]

pesi_tmp_d[1][0] = pesi_archi[0][0]
pesi_tmp_d[1][1] = pesi_archi[0][1]
pesi_tmp_d[1][2] = pesi_archi[0][2]

In [ ]:
model_d.set_weights(pesi_tmp_d)
history_d = model_d.fit(trainX_d, trainY_d, epochs=3)
score_d = model_d.evaluate(testX_d, testY_d, verbose=0)